In [1]:
# ipython extension that will alow us to benchmark execution time for the entire cell in notebook
!pip install ipython-autotime

In [2]:
import numpy as np
import pandas as pd
from time import time

In [3]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/1ozmJBOaE2Xh6gjDGqy4TI27XIhyAL6h60QRy1MUsDso/pub?output=csv')
df.head(5)

,Date,Adj Close AAPL,Adj Close ABC,Adj Close AMZN,Adj Close BAC,Adj Close BP,Adj Close BRK-B,Adj Close CVS,Adj Close DDAIF,Adj Close F,Adj Close GOOG,Adj Close JPM,Adj Close MCK,Adj Close MSFT,Adj Close RDS-B,Adj Close T,Adj Close TM,Adj Close TOT,Adj Close UNH,Adj Close WMT,Adj Close XOM,Volume AAPL,Volume ABC,Volume AMZN,Volume BAC,Volume BP,Volume BRK-B,Volume CVS,Volume DDAIF,Volume F,Volume GOOG,Volume JPM,Volume MCK,Volume MSFT,Volume RDS-B,Volume T,Volume TM,Volume TOT,Volume UNH,Volume WMT,Volume XOM
0,2005-02-21,1.368799,11.901972,34.990002,32.976032,28.452970,60.320000,19.241392,31.138027,8.256411,92.587959,24.088234,33.270592,18.141571,31.159843,10.165306,59.931969,24.774008,38.530373,36.302223,37.982330,4999013600,22080000,31627600,37366400,18590600,2125000,12465000,1408800,29261000,130201203,45331800,5752300,327812600,1838189,36831500,915300,8115000,21417000,46536700,108879400
1,2005-02-28,1.316963,12.427639,35.849998,32.968975,29.017036,59.580002,20.286993,30.857567,7.875347,92.602905,24.440075,33.685661,18.084099,32.294121,10.282875,60.843304,25.259737,39.331413,37.437328,38.168442,3746232000,20682800,29684900,48255000,20998700,3420000,34757800,1908600,55751500,77886448,52037600,9939700,322103600,2484741,33702100,1086900,11717200,20941800,61478000,128760400
2,2005-03-07,1.238824,11.963464,34.750000,32.568451,28.190603,60.020000,20.391163,30.957735,7.868996,88.568024,23.527895,33.156551,18.026617,31.413136,10.077134,60.897354,25.029329,38.062389,36.379776,36.655407,4204488400,20746800,29673700,41283300,20307100,4090000,20628200,1993600,41372100,94478205,60048400,8092000,316021800,2355602,30300300,1033500,12568800,23441200,47133000,170203800
3,2005-03-14,1.321578,12.072557,34.160000,31.977856,28.584154,58.369999,20.391163,30.049568,7.233888,89.683846,23.462727,32.680378,17.466204,31.099272,9.804211,59.136475,25.292948,38.471340,36.273994,37.616077,3553877600,18499200,26026800,55968800,15497500,5540000,17063800,3556300,79130600,88381237,62250200,5431700,408338500,2058276,49319000,1251700,8275800,25304200,152631200,112404900
4,2005-03-21,1.307426,12.122155,32.880001,31.131090,26.983763,57.900002,20.622667,29.655588,7.170377,89.290321,22.759045,33.553383,17.444654,29.518976,9.783216,57.777191,24.109774,39.145905,35.822327,35.424568,2055071200,8516800,23236100,50533000,15531900,2980000,14253200,2137700,58470600,43493615,55754800,5021800,331673700,1687673,33866900,1247600,7194000,19556000,70161800,88667200


In [4]:
# Choose as X `Volume` columns and then convert them to numpy array
X = df.filter(regex='Volume').to_numpy()
X

array([[4999013600,   22080000,   31627600, ...,   21417000,   46536700,
         108879400],
       [3746232000,   20682800,   29684900, ...,   20941800,   61478000,
         128760400],
       [4204488400,   20746800,   29673700, ...,   23441200,   47133000,
         170203800],
       ...,
       [ 345301700,    5329400,   13237600, ...,   18515000,   25487800,
         138395500],
       [ 362298700,    3276200,   13185300, ...,   11655700,   62243200,
          96610400],
       [  87668834,     890390,    4305184, ...,    2315864,   12203132,
          22488620]])

In [5]:
%load_ext autotime

time: 120 µs (started: 2022-02-16 07:58:24 +00:00)


We need to compare two times. With and without QR. 

In [6]:
X_pinverse = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T)

time: 12 ms (started: 2022-02-16 07:58:24 +00:00)


In [7]:
Q, R = np.linalg.qr(X)
X_pinverse_qr = np.dot(np.linalg.inv(R), Q.T)

time: 27.8 ms (started: 2022-02-16 07:58:24 +00:00)


In [8]:
%unload_ext autotime

Here one time it gave on colab 16.8 ms without and 4.63 with QR. On the other hand, on local machine 46 ms standard and 90 ms with QR. Perhaps, these measurements show random noise due to small matrix sizes. The second run of Colab gives 23.9 while with QR it is 20.9. The third time gave 20.6 simple vs 3.1 with QR. 

In [9]:
np.sum(np.isclose(X_pinverse, X_pinverse_qr))

16720

In [10]:
np.sum(np.allclose(X_pinverse, X_pinverse_qr))

1

All the elements are close. 

In [11]:
X_pinverse_qr.shape

(20, 836)

In [12]:
np.prod(X_pinverse_qr.shape)

16720